In [31]:
import numpy as np
import pandas as pd
import random
import os
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score,precision_score, recall_score, average_precision_score,cohen_kappa_score,matthews_corrcoef
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import   Dense, Dropout,Conv2D,Flatten,Conv1D,TimeDistributed,LSTM,AveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [40]:
# import training dataset
df_train = pd.read_csv(r"/content/training_1416.csv")
df_train.head()

,rcvTime,sendTime,sender,senderPseudo,messageID,pos_x,pos_y,spd_x,spd_y,acl_x,acl_y,hed_x,hed_y,receiver_id,nttack
0,50408.087237,50408.087237,27.0,10275.0,16969.0,223.005975,364.802772,-0.079871,0.537359,-0.388361,2.612909,-0.14702,0.989133,21,0.0
1,50408.392900,50408.392900,21.0,10215.0,17039.0,220.358432,383.935288,-0.078168,0.525905,-0.388913,2.616645,-0.14702,0.989133,27,0.0
2,50409.087237,50409.087237,27.0,10275.0,17121.0,222.757252,366.657904,-0.419586,2.822917,-0.360449,2.425112,-0.14702,0.989133,21,0.0
3,50409.392900,50409.392900,21.0,10215.0,17151.0,220.152543,385.761996,-0.421901,2.838498,-0.322463,2.169573,-0.14702,0.989133,27,0.0
4,50410.087237,50410.087237,27.0,10275.0,17193.0,222.453841,370.546522,-0.695096,4.676514,-0.330370,2.222776,-0.14144,0.989947,21,0.0


In [41]:
# import test dataset
df_test =pd.read_csv(r"/content/test_1416.csv")
df_test.head()

,rcvTime,sendTime,sender,senderPseudo,messageID,pos_x,pos_y,spd_x,spd_y,acl_x,acl_y,hed_x,hed_y,receiver_id,nttack
0,55646.969136,55646.969136,7335.0,2073355.0,22841012.0,226.535204,219.450184,2.866232,0.415503,2.982118,0.336361,0.999531,0.030620,7719,1.0
1,55646.988357,55646.988357,7653.0,5076535.0,22841125.0,222.391845,346.367786,0.473989,-3.036948,0.451540,-3.041015,-0.044535,0.999008,7695,1.0
2,55646.988357,55646.988357,7653.0,5076535.0,22841125.0,222.391845,346.367786,0.473989,-3.036948,0.451540,-3.041015,-0.044535,0.999008,7689,1.0
3,55646.988357,55646.988357,7653.0,5076535.0,22841125.0,222.391845,346.367786,0.473989,-3.036948,0.451540,-3.041015,-0.044535,0.999008,7683,1.0
4,55646.988357,55646.988357,7653.0,5076535.0,22841125.0,222.391845,346.367786,0.473989,-3.036948,0.451540,-3.041015,-0.044535,0.999008,7677,1.0


In [42]:
# Creating training and test sets
x_train = df_train.drop(['nttack','receiver_id','sender'],axis=1).values
y_train = df_train['nttack'].values

x_test = df_test.drop(['nttack','receiver_id','sender'],axis=1).values
y_test = df_test['nttack'].values
#Normalizing the data
sc = StandardScaler()
x_train  = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
# reshaping the target variable
y_test = y_test.reshape(-1,1)
y_train = y_train.reshape(-1,1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [43]:
# reshaping the data for CONV1D input
reshaped_x = x_train.reshape(x_train.shape[0],12,1)
# verifying the new shape
reshaped_x.shape

(1130329, 12, 1)

In [44]:
# model creation
model = Sequential()
model.add(Conv1D(32, kernel_size=(3),activation='relu',input_shape=(12,1),padding='same'))
model.add(AveragePooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=(3),activation='relu'))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(64,return_sequences =True))
model.add(LSTM(128,return_sequences =True))
model.add(LSTM(256))
model.add(Dense(64,activation='relu'))
model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_40 (Conv1D)          (None, 12, 32)            128       
                                                                 
 average_pooling1d_6 (Averag  (None, 6, 32)            0         
 ePooling1D)                                                     
                                                                 
 conv1d_41 (Conv1D)          (None, 4, 64)             6208      
                                                                 
 time_distributed_40 (TimeDi  (None, 4, 64)            0         
 stributed)                                                      
                                                                 
 lstm_18 (LSTM)              (None, 4, 64)             33024     
                                                                 
 lstm_19 (LSTM)              (None, 4, 128)          

In [45]:
# model compilation
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# model training
hist = model.fit(reshaped_x,y_train,epochs=10,batch_size=8)

Epoch 1/10
141292/141292 [==============================] - 4325s 31ms/step - loss: 0.3475 - accuracy: 0.8288
Epoch 2/10
141292/141292 [==============================] - 4456s 32ms/step - loss: 0.3334 - accuracy: 0.8342
Epoch 3/10
141292/141292 [==============================] - 4524s 32ms/step - loss: 0.3164 - accuracy: 0.8429
Epoch 4/10
141292/141292 [==============================] - 4442s 31ms/step - loss: 0.3047 - accuracy: 0.8500
Epoch 5/10
141292/141292 [==============================] - 4440s 31ms/step - loss: 0.2940 - accuracy: 0.8569
Epoch 6/10
141292/141292 [==============================] - 4351s 31ms/step - loss: 0.2886 - accuracy: 0.8605
Epoch 7/10
141292/141292 [==============================] - 4842s 34ms/step - loss: 0.2854 - accuracy: 0.8631
Epoch 8/10
141292/141292 [==============================] - 4303s 30ms/step - loss: 0.2852 - accuracy: 0.8634
Epoch 9/10
 37571/141292 [======>.......................] - ETA: 53:54 - loss: 0.2840 - accuracy: 0.8636